In [1]:
#!/usr/bin/env python3
'''
monitor your last 5 GO task_id's

Based on tutorial and documentation at: 
   http://globus.github.io/globus-sdk-python/index.html

-Galen Arnold, 2017, NCSA
'''

import time
import globus_sdk

CLIENT_ID='2826c2c1-2bdf-4ea4-8f3a-d25487ad9c4c'
client = globus_sdk.NativeAppAuthClient(CLIENT_ID)
client.oauth2_start_flow_native_app(refresh_tokens=True)

authorize_url = client.oauth2_get_authorize_url()
print('Please go to this URL and login: {0}'.format(authorize_url))

# this is to work on Python2 and Python3 -- you can just use raw_input() or
# input() for your specific version
get_input = getattr(__builtins__, 'raw_input', input)
auth_code = get_input(
    'Please enter the code you get after login here: ').strip()
token_response = client.oauth2_exchange_code_for_tokens(auth_code)
globus_auth_data = token_response.by_resource_server['auth.globus.org']

Please go to this URL and login: https://auth.globus.org/v2/oauth2/authorize?client_id=2826c2c1-2bdf-4ea4-8f3a-d25487ad9c4c&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=openid+profile+email+urn%3Aglobus%3Aauth%3Ascope%3Atransfer.api.globus.org%3Aall&state=_default&response_type=code&code_challenge=Feh023Wwili7khzFCw9qQE-1MSw-yqrxSy1dvt7wDmM%3D&code_challenge_method=S256&access_type=offline
Please enter the code you get after login here: M6nZZwl87cxmaObFlRJUS4CPIgC9HH


In [2]:
def my_task_list(token_response):
	globus_transfer_data = token_response.by_resource_server['transfer.api.globus.org']
	# the refresh token and access token, often abbr. as RT and AT
	transfer_rt = globus_transfer_data['refresh_token']
	transfer_at = globus_transfer_data['access_token']
	expires_at_s = globus_transfer_data['expires_at_seconds']

	# a GlobusAuthorizer is an auxiliary object we use to wrap the token. In
	# more advanced scenarios, other types of GlobusAuthorizers give us
	# expressive power

	authorizer = globus_sdk.RefreshTokenAuthorizer(
	transfer_rt, client, access_token=transfer_at, expires_at=expires_at_s)

	tc = globus_sdk.TransferClient(authorizer=authorizer)

	for transfer in tc.task_list(num_results=5):
    		print("task_id= ( %s ) %s -> %s" % 
                  (transfer['task_id'],
                   transfer['source_endpoint_display_name'],
                   transfer['destination_endpoint_display_name']))
    		print("request_time %s" % transfer['request_time'])
    		print("completed    %s" % transfer['completion_time'])
    		print("type=%s Mbytes=%.1f Mbytes/s=%.1f" % 
                  (transfer['type'],transfer['bytes_transferred']/1024,
                   transfer['effective_bytes_per_second']/1024))
	    	print("=====================================================")

	print("______________________________________________________________________________")
	time.sleep(30)
# end def my_task_list

In [3]:
while (True):
    my_task_list(token_response)
# end while    

task_id= ( e714a6ac-05ae-11e7-bb08-22000b9a448b ) ncsa#BlueWaters -> None
request_time 2017-03-10 16:30:42+00:00
completed    2017-03-10 16:30:49+00:00
type=DELETE Mbytes=0.0 Mbytes/s=0.0
task_id= ( 5fc769fa-0509-11e7-bb05-22000b9a448b ) ncsa#BlueWaters -> ncsa#jyc
request_time 2017-03-09 20:45:48+00:00
completed    2017-03-09 21:28:40+00:00
type=TRANSFER Mbytes=283330443.4 Mbytes/s=110183.7
task_id= ( 83f1c9fa-f9d4-11e6-ba9e-22000b9a448b ) ncsa#BlueWaters -> ncsa#jyc
request_time 2017-02-23 14:29:43+00:00
completed    2017-02-23 14:29:50+00:00
type=TRANSFER Mbytes=12219.0 Mbytes/s=1624.0
task_id= ( 41978644-f9d4-11e6-ba9e-22000b9a448b ) ncsa#BlueWaters -> ncsa#jyc
request_time 2017-02-23 14:27:51+00:00
completed    2017-02-23 14:27:59+00:00
type=TRANSFER Mbytes=12219.0 Mbytes/s=1585.8
task_id= ( 9f5467a2-e192-11e6-9d3a-22000a1e3b52 ) arnoldg#macos -> ncsa#BlueWaters
request_time 2017-01-23 17:37:34+00:00
completed    2017-01-23 17:37:38+00:00
type=TRANSFER Mbytes=87478.0 Mbytes/s=1972

KeyboardInterrupt: 